In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.generation.utils import GenerationConfig

model_name = "baichuan-inc/Baichuan2-7B-Chat"

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype=torch.bfloat16, trust_remote_code=True)
model.generation_config = GenerationConfig.from_pretrained(model_name)

prompts = [
    "中国的首都是哪里？",
    "中国的首都是哪裡？",
    "中国 的首都是哪里？",
    "中国的首都是那里？",
    "中国的首都是哪？",
    "中国的capital是哪？",
    "中国的首府是哪里？",
    "Where is the capital of China?",
]

for prompt in prompts:
    messages = []
    messages.append({"role": "user", "content": prompt})
    response = model.chat(tokenizer, messages)
    print(f"输入：{prompt}\n输出：{response.strip()}\n")

输入：中国的首都是哪里？
输出：中国的首都是北京。

输入：中国的首都是哪裡？
输出：中国的首都是北京。

输入：中国 的首都是哪里？
输出：中国的首都是北京。

输入：中国的首都是那里？
输出：中国的首都是北京。

输入：中国的首都是哪？
输出：中国的首都是北京。

输入：中国的capital是哪？
输出：中国的首都，也就是政治中心是北京。而经济中心则是上海。

输入：中国的首府是哪里？
输出：中国的首都为北京。

输入：Where is the capital of China?
输出：The capital of China is Beijing.



SQuAD2.0是斯坦福问答数据集（Stanford Question Answering Dataset），它是在 1.0是斯坦福大学发布的一个广泛用于机器阅读理解与问答（Question Answering, QA）研究的标准数据集。SQuAD v2 是 SQuAD v1.1 的升级版本，专门用于评估模型“在自然文本中寻找答案”的能力，以及判断“问题是否无法回答”的鲁棒性。数据集中的全部问题均基于英文维基百科文章，由人工编写。SQuAD v2 与 v1.1 最大的区别是引入了大量“无答案”问题，使模型不仅要抽取答案，还要判断问题是否可解，这极大提升了任务难度和鲁棒性评测价值。

In [9]:
from datasets import load_dataset
import random
import pandas as pd

# 加载 SQuAD v2 数据集
dataset = load_dataset("squad_v2")

# 1. 展示数据规模
train_size = len(dataset["train"])
val_size = len(dataset["validation"])

print(f"SQuAD v2 训练集样本数: {train_size}")
print(f"SQuAD v2 验证集样本数: {val_size}")

# 2. 展示一个数据样例，并介绍结构
sample = dataset["train"][0]
sample


SQuAD v2 训练集样本数: 130319
SQuAD v2 验证集样本数: 11873


{'id': '56be85543aeaaa14008c9063',
 'title': 'Beyoncé',
 'context': 'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".',
 'question': 'When did Beyonce start becoming popular?',
 'answers': {'text': ['in the late 1990s'], 'answer_start': [269]}}

为了直观展示SQuAD v2数据集的结构，我们展示一个数据样例，并介绍数据集的结构。

In [11]:
# 转成 DataFrame 展示更直观
df = pd.DataFrame({
    "title": [sample["title"]],
    "context": [sample["context"]],
    "question": [sample["question"]],
    "answers_text": [sample["answers"]["text"]],
    "answers_start": [sample["answers"]["answer_start"]],
    "id": [sample["id"]]
})
display(df)

# 3. 对数据样例结构的说明
print("\n数据样例结构说明：")
print("title          ：维基百科文章标题")
print("context        ：一段来自维基百科的上下文文本")
print("question       ：与该段上下文相关的自然语言提问")
print("answers_text   ：标准答案，可能为包含在context中的文本片段；如该问题无答案，则为空列表")
print("answers_start  ：答案在context中起始字符位置列表")
print("id             ：每条样本的唯一标识符")

,title,context,question,answers_text,answers_start,id
0,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce start becoming popular?,[in the late 1990s],[269],56be85543aeaaa14008c9063



数据样例结构说明：
title          ：维基百科文章标题
context        ：一段来自维基百科的上下文文本
question       ：与该段上下文相关的自然语言提问
answers_text   ：标准答案，可能为包含在context中的文本片段；如该问题无答案，则为空列表
answers_start  ：答案在context中起始字符位置列表
id             ：每条样本的唯一标识符
